In [2]:
import sys
sys.path.append('../')
import transport_frames.src.graph_builder.criteria as criteria
import transport_frames.src.metrics.indicators as indicators
import transport_frames.src.metrics.grade_territory as grade_territory
import transport_frames.src.graph_builder.graphbuilder as graphbuilder
import momepy
import osmnx as ox
import geopandas as gpd
import shapely
import pandas as pd
import networkx as nx
import numpy as np
import json
from shapely import wkt


/Users/sashamorozov/Documents/Code/NCCR/transport_frames/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# pipeline

In [3]:
region_name = 'Тульская_область'
local_crs = 32637

In [21]:
# Считывание входных данных

PLACEHOLDER = gpd.GeoDataFrame(geometry=[]) # если нет данных о сервисе

# полигоны районов и муниципальных образований
settlement = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_settlement.geojson')
district = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_district.geojson')

# населенные пункты и админ.центры точками
region_points = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_region_points.geojson')
admin_centers = region_points[region_points['is_admin_centre_district']==True]

# Сервисы
railway_stations = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_railway_stations.geojson')
railway_roads = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_railway_roads.geojson')
bus_stops = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_bus_stops.geojson')
bus_routes = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_bus_routes.geojson')
fuel_stations = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_fuel_stations.geojson')
ferry_terminal = PLACEHOLDER
local_aerodrome = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_aerodrome.geojson')
local_aerodrome['geometry'] = shapely.centroid(local_aerodrome['geometry']).set_crs(local_aerodrome.crs)
# international_aerodrome = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_international_aerodrome.geojson')
# либо:
international_aerodrome = local_aerodrome[local_aerodrome['aerodrome:type']=='international']
international_aerodrome['geometry'] = shapely.centroid(international_aerodrome['geometry']).set_crs(international_aerodrome.crs)

# Природные объекты
water = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_water.geojson')
nature_reserve = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_nature_reserve.geojson')

# Кастомная территория
sasha = district.iloc[[3]].to_crs(local_crs)
custom_territory = sasha
custom_territory_center = custom_territory.geometry.representative_point()
custom_territory_center = gpd.GeoDataFrame([{'geometry': custom_territory_center.iloc[0]}], crs=custom_territory_center.crs).to_crs(local_crs)


In [7]:
sasha.crs

<Projected CRS: EPSG:32637>
Name: WGS 84 / UTM zone 37N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 36°E and 42°E, northern hemisphere between equator and 84°N, onshore and offshore. Djibouti. Egypt. Eritrea. Ethiopia. Georgia. Iraq. Jordan. Kenya. Lebanon. Russian Federation. Saudi Arabia. Somalia. Sudan. Syria. Türkiye (Turkey). Ukraine.
- bounds: (36.0, 0.0, 42.0, 84.0)
Coordinate Operation:
- name: UTM zone 37N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [12]:
region_capital = ox.geocode_to_gdf('N34389350',by_osmid=True)
city = ox.geocode_to_gdf(f'R{81993}', by_osmid=True).to_crs(epsg=4326)
city['layer'] = region_name
city['status'] = 'region'

In [22]:
# Загрузка УДС графа из файла 

citygraph = nx.read_graphml(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/graphs/{region_name}/{region_name}_uds.graphml')
citygraph = graphbuilder.convert_list_attr_from_str(citygraph)
citygraph = indicators.prepare_graph(citygraph)

In [14]:
# Загрузка интермодального графа из файла 

inter = nx.read_graphml(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/graphs/{region_name}/{region_name}_inter.graphml')
inter = graphbuilder.convert_list_attr_from_str(inter)
inter = indicators.prepare_graph(inter)

In [23]:
citygraph_n,citygraph_e = momepy.nx_to_gdf(citygraph)

In [24]:
services = {
    'railway_stops' : railway_stations,
    'fuel' : fuel_stations,
    'ports' : ferry_terminal,
    'local_aero' : local_aerodrome,
    'international_aero' : international_aerodrome,
    'oopt' : nature_reserve,
    'water_objects' : water,
    'train_paths' : railway_roads,
    'bus_stops' : bus_stops,
    'capital': region_capital,
    'points': region_points,
    'bus_routes' : bus_routes,
    'region_centers': admin_centers,
    'neud_center': custom_territory_center
}

In [8]:
# services = {
#     'railway_stops' : railway_stations, # ЖД остановки
#     'fuel' : fuel_stations, # АЗС станции
    
    
#     'train_paths' : railway_roads, # ЖД пути
    
    
#     'local_aero' : local_aerodrome, # Аэропорты местного значения
#     'international_aero' : international_aerodrome, # Аэропорты международного значения
#     'bus_stops' : bus_stops, # Автобусные остановки 
#     'bus_routes' : bus_routes, # Автобусные маршруты
#     'capital': region_capital, # Столица региона (точка)
#     'points': region_points, # Населенные пункты региона (точки)
#     'region_centers': admin_centers, # Административные центры муниципальных районов региона (точки)
#     'oopt' : nature_reserve, # Особо охраняемые природные объекты
#     'water_objects' : water, # Водные объекты
#     'neud_center': custom_territory_center # Центроида полигона кастомной территории (территория+3 км буфер)
# }

In [25]:
G = indicators.assign_services_names_to_nodes(services,citygraph_n,citygraph,crs = local_crs)

railway_stops
fuel
no ports (((((((
local_aero
no international_aero (((((((
no oopt (((((((
no water_objects (((((((
no train_paths (((((((
bus_stops
capital
points
bus_routes
region_centers
neud_center


In [10]:
merged = indicators.indicator_area(G,inter,services,[city,settlement,district],local_crs)

100%|██████████| 2871/2871 [04:49<00:00,  9.91it/s]


new version


/Users/sashamorozov/Documents/Code/NCCR/transport_frames/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)


new version


/Users/sashamorozov/Documents/Code/NCCR/transport_frames/transport_frames/src/metrics/indicators.py:779: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  merged[cols_to_format] = merged[cols_to_format].applymap(
/Users/sashamorozov/Documents/Code/NCCR/transport_frames/transport_frames/src/metrics/indicators.py:301: UserWarning: `keep_geom_type=True` in overlay resulted in 4 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  roads_intersect = gpd.overlay(roads_gdf, polygons_gdf, how='intersection')
/Users/sashamorozov/Documents/Code/NCCR/transport_frames/transport_frames/src/metrics/indicators.py:311: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result = polygons_gdf.merge

In [12]:
merged[2]

,index,name,status,layer,geometry,fuel_stations_accessibility,train_stops_accessibility,local_aero_accessibility,international_aero_accessibility,ports_accessibility,...,number_of_fuel_stations,number_of_train_stops,number_of_local_aero,number_of_international_aero,number_of_ports,number_of_bus_stops,reg1_length,reg2_length,reg3_length,number_of_bus_routes
0,0,Арсеньевский район,муниципальный район,Арсеньевский муниципальный район,"POLYGON ((36.27046 53.67571, 36.27343 53.67505...",19.453,36.541,146.323,None,None,...,1.000,0.000,0.000,0.000,0.000,8.000,0.000,308.381,493.096,0.000
1,1,Белевский район,муниципальный район,Белевский муниципальный район,"POLYGON ((35.89856 53.85018, 35.90015 53.84679...",14.032,16.958,129.721,None,None,...,3.000,1.000,0.000,0.000,0.000,48.000,111.578,193.823,952.544,0.000
2,2,Богородицкий район,муниципальный район,Борогодицкий муниципальный район,"POLYGON ((37.86240 53.73471, 37.86404 53.72565...",12.263,18.803,90.468,None,None,...,13.000,2.000,0.000,0.000,0.000,79.000,109.024,337.189,700.363,0.000
3,3,Веневский район,муниципальный район,Веневский муниципальный район,"POLYGON ((37.86087 54.34286, 37.86265 54.34249...",7.957,15.425,65.040,None,None,...,26.000,2.000,0.000,0.000,0.000,116.000,251.574,279.923,1310.774,1.000
4,4,Воловский район,муниципальный район,Воловский муниципальный район,"POLYGON ((37.73668 53.45977, 37.74516 53.45518...",13.393,14.224,50.066,None,None,...,6.000,2.000,0.000,0.000,0.000,29.000,64.398,244.293,798.714,0.000
5,5,Алексин,городской округ,городской округ Алексин,"POLYGON ((36.85710 54.47946, 36.86016 54.47653...",11.520,16.729,49.543,None,None,...,10.000,4.000,0.000,0.000,0.000,132.000,31.591,540.871,976.713,23.000
6,6,Донской,городской округ,городской округ Донской,"POLYGON ((38.24402 53.92843, 38.24521 53.92841...",4.644,9.539,98.191,None,None,...,7.000,1.000,0.000,0.000,0.000,67.000,0.000,53.867,385.137,0.000
7,7,Ефремов,городской округ,городской округ Ефремов,"POLYGON ((37.81869 53.07204, 37.82000 53.06810...",10.444,21.743,26.653,None,None,...,16.000,3.000,1.000,0.000,0.000,135.000,83.385,509.899,1207.673,0.000
8,8,Новогуровский,городской округ,городской округ Новогуровский,"POLYGON ((37.30758 54.48843, 37.31015 54.48584...",1.661,5.482,35.776,None,None,...,1.000,0.000,0.000,0.000,0.000,2.000,0.000,8.433,26.095,0.000
9,9,Новомосковск,городской округ,городской округ Новомосковск,"POLYGON ((38.09889 54.09169, 38.10352 54.08043...",7.048,13.607,84.265,None,None,...,21.000,12.000,0.000,0.000,0.000,215.000,59.490,383.756,889.316,20.000


In [27]:
temp = indicators.indicator_territory(G,custom_territory,local_crs,district,region_points,admin_centers,services,*momepy.nx_to_gdf(G))

100%|██████████| 1/1 [00:00<00:00,  8.09it/s]


nearest_region_centers
nearest_settlement_centers


In [28]:
temp

,name,to_region_admin_center,to_reg1,fuel_stations_accessibility,train_stops_accessibility,local_aero_accessibility,international_aero_accessibility,ports_accessibility,water_objects_accessibility,oopt_accessibility,...,number_of_international_aero,number_of_bus_stops,number_of_water_objects,number_of_oopt,density,train_path_length,number_of_bus_routes,to_nearest_settlement_center,to_nearest_mo_center,geometry
7910,None,49.587,5.376,0,0,70.714,None,None,0.008,0.554,...,0,116,674,0,1132.591,128.61,63,762.822,7518.282,"POLYGON ((37.86087 54.34286, 37.86265 54.34249..."
